In [2]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

# 라이브러리 임포트 
import os
import numpy as np
import pandas as pd

import time
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import urllib.request

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# gp 설정
%matplotlib inline
plt.rcParams['font.size'] = 11.0
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'retina'

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

### 사용 모듈
- requests 모듈 : 직접 주소를 입력하여 요청할 수 있다. requests 모듈을 통해 받아온 데이터는 웹 브라우저에서 소스보기를 했을때 나오는 코드이다. <br/>따라서 requests 모듈을 통해 받아온 데이터에는 브라우저 화면에 있는 데이터가 없을 수도 있다.
- selenium : 웹 브라우저를 조작하여 원하는 페이지로 이동한 다음에 데이터를 받아온다. 크롬 개발자 도구의 elements 탭에 있는 데이터를 받아올 수 있다.
- bs4 : 받아온 html, xml 데이터를 분석하여 수집한다.
- json : 받아온 json데이터를 분석하여 수집한다.
- feedparser : 받아온 rss데이터를 분석하여 수집한다.

### 웹 문서 수집시 알아야할 사항
- 요청할 주솟 분석이 가능하다면 requests 모듈로 요청한다.
- 요청할 주소 분석이 어렵다면 selenium을 사용한다.
- requests 모듈은 수집시간이 짧고 selsenium은 수집시간이 길어진다.
- 소스보기 했을 때 데이터가 있다면 : requests 모듈로 요청하고 bs4로 수집한다.
- 소스보기 했을 때 데이터가 없고 elements 탭에 데이터가 있을 경우 : ajax인가, 웹소켓인가...
- ajax인 경우 : 요청 주소를 파악하여 requests로 요청한다.
- iframe인 경우 : iframe 태그의 src속성의 주소를 파악하여 requests로 요청한다.
- 웹 소켓인 경우 : selenium을 통해 데이터를 수집한다.
- 소스보기 했을때 데이터가 없고 elements 탭에서도 데이터가 없을 경우 : iframe
- iframe이 있는지 확인해보고 있다면 src 속성의 주소를 통해 requests 모듈로 요청한다.

### 태그를 지칭하기 위해 알아야할 것들
- 원하는 데이터를 가지고 있는 태그를 지칭할 수 있어야 필요한 데이터를 수집할 수 있다.
- id 속성 : html에서 사용하는 모든 태그에 사용할 수 있는 속성. 이 속성은 하나의 html 문서안에 같은 id를 가지고 있는 태그는 없다. <br/>id는 웹 개발자가 특정 태그 하나를 지칭하기 위해 사용한다. 우리는 데이터를 가지고 있는 특정 태그를 지칭하는 용도로 사용할 수 있다.
- class 속성 : css는 html 태그를 꾸미는 용도로 사용한다. 이러한 css들을 미리 정의해놓고 태그에 적용할 때 가용하는것이 class 속성이다. <br/>웹을 개발할 때 특정 영역의 모양을 설정하기 위해 그 영역만을 위한 css를 만들어 사용하는 경우가 있다. 만약 우리가 필요로 하는 데이터가 특정 영역에 있고 그 영역에서만 사용하는 class 속성이 있다면 이를 통해 접근할 수 있다.
- nth-child : 같은 태그가 동시에 여러개 나열되어 있는 경우 그 순서를 구분하기 위해 사용한다(1부터 시작)<br/>여러 항목들을 가져오고자 할 때는 그 항목을 둘러 싸고 있는 태그부터 가져온다.

### 코드 작성 방법
- 한 페이지의 데이터를 수집한다.
- 다음 페이지로 넘어갈 수 있는 방법을 찾는다.
- 마지막 페이지 여부를 어떻게 확인할 수 있는 방법을 찾는다.
- while 문을 만들어 마지막 페이지까지 갈 수 있도록 구성한다.
- 반드시 수집을 위해 만든 프로그램이 종료가 되게 만들어야한다(while문의 종료)
- 매 요청시마다 딜레이를 줘야 한다(1~3초)